你可以在任意的路径操作中使用 response_model 参数来声明用于响应的模型：

@app.get()
@app.post()
@app.put()
@app.delete()

注意，response_model是「装饰器」方法（get，post 等）的一个参数。不像之前的所有参数和请求体，它不属于路径操作函数。
等等。

它接收的类型与你将为 Pydantic 模型属性所声明的类型相同，因此它可以是一个 Pydantic 模型，

但也可以是一个由 Pydantic 模型组成的 list，例如 List[Item]。

FastAPI 将使用此 response_model 来：

将输出数据转换为其声明的类型。
校验数据。
在 OpenAPI 的路径操作中为响应添加一个 JSON Schema。
并在自动生成文档系统中使用。
但最重要的是：

会将输出数据限制在该模型定义内。下面我们会看到这一点有多重要。

响应模型在参数中被声明，而不是作为函数返回类型的注解，这是因为路径函数可能不会真正返回该响应模型，而是返回一个 dict、数据库对象或其他模型，然后再使用 response_model 来执行字段约束和序列化。

我们可以创建一个有明文密码的输入模型和一个没有明文密码的输出模型：

In [2]:
import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel, EmailStr

app = FastAPI()

class UserIn(BaseModel):
    username: str
    password: str
    email: EmailStr
    full_name: str | None = None


class UserOut(BaseModel):
    username: str
    email: EmailStr
    full_name: str | None = None


@app.post("/user/", response_model=UserOut)
async def create_user(user: UserIn):
    return user

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [11768]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:59846 - "POST /user/ HTTP/1.1" 422 Unprocessable Entity
INFO:     127.0.0.1:59850 - "POST /user/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [11768]


In [ ]:
FastAPI 将会负责过滤掉未在输出模型中声明的所有数据（使用 Pydantic）。

## requests库

In [ ]:
url = 'http://127.0.0.1:8009/user/' 
data = {
    "username": "Foo",
    "password": "1234321",
    "email": "example@example.com"
}
res = requests.post(url, json=data) 
res.text

# 在另一个ipynb文件中运行代码，会得到 
# '{"username":"Foo","email":"example@example.com","full_name":null}'

## 响应模型编码参数

你的响应模型可以具有默认值，例如：

In [5]:
import uvicorn
from typing import List
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float = 10.5
    tags: List[str] = []


items = {
    "foo": {"name": "Foo", "price": 50.2},
    "bar": {"name": "Bar", "description": "The bartenders", "price": 62, "tax": 20.2},
    "baz": {"name": "Baz", "description": None, "price": 50.2, "tax": 10.5, "tags": []},
}


@app.get("/items/{item_id}", response_model=Item, response_model_exclude_unset=True)
async def read_item(item_id: str):
    return items[item_id]

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [11768]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:60216 - "GET /items/foo HTTP/1.1" 200 OK
INFO:     127.0.0.1:60221 - "GET /items/bar HTTP/1.1" 200 OK
INFO:     127.0.0.1:60234 - "GET /items/baz HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [11768]


description: Union[str, None] = None 具有默认值 None。

tax: float = 10.5 具有默认值 10.5.

tags: List[str] = [] 具有一个空列表作为默认值：

但如果它们并没有存储实际的值，你可能想从结果中忽略它们的默认值。

你可以设置路径操作装饰器的 response_model_exclude_unset=True 参数：

然后响应中将不会包含那些默认值，而是仅有实际设置的值。

因此，如果你向路径操作发送 ID 为 foo 的商品的请求，则响应（不包括默认值）将为：

{
    "name": "Foo",
    "price": 50.2
}

In [ ]:
import requests
url = 'http://127.0.0.1:8009/items/foo' 
res = requests.get(url) 
res.text

# 在另一个ipynb文件中运行代码，会得到 '{"name":"Foo","price":50.2}'

但是，如果你的数据在具有默认值的模型字段中有实际的值，例如 ID 为 bar 的项：

{
    "name": "Bar",
    "description": "The bartenders",
    "price": 62,
    "tax": 20.2
}

In [ ]:
url = 'http://127.0.0.1:8009/items/bar' 
res = requests.get(url) 
res.text

# 在另一个ipynb文件中运行代码，会得到 '{"name":"Bar","description":"The bartenders","price":62.0,"tax":20.2}'

如果数据具有与默认值相同的值，例如 ID 为 baz 的项：

{
    "name": "Baz",
    "description": None,
    "price": 50.2,
    "tax": 10.5,
    "tags": []
}

In [ ]:
url = 'http://127.0.0.1:8009/items/baz' 
res = requests.get(url) 
res.text

# 在另一个ipynb文件中运行代码，会得到 '{"name":"Baz","description":null,"price":50.2,"tax":10.5,"tags":[]}'

即使 description、tax 和 tags 具有与默认值相同的值，FastAPI 足够聪明 (实际上是 Pydantic 足够聪明) 去认识到这一点，它们的值被显式地所设定（而不是取自默认值）。

因此，它们将包含在 JSON 响应中。

你还可以使用：

response_model_exclude_defaults=True
response_model_exclude_none=True

使用路径操作装饰器的 response_model 参数来定义响应模型，特别是确保私有数据被过滤掉。

使用 response_model_exclude_unset 来仅返回显式设定的值。